In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

def load_df(csv_path='D:\\Documents\\ga-customer-revenue-prediction\\test_v2.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df


df_small = load_df(nrows=100000)


Loaded test_v2.csv. Shape: (100000, 59)


In [2]:
df_small.columns

Index(['channelGrouping', 'customDimensions', 'date', 'fullVisitorId', 'hits',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'device.browser', 'device.browserSize', 'device.browserVersion',
       'device.deviceCategory', 'device.flashVersion', 'device.isMobile',
       'device.language', 'device.mobileDeviceBranding',
       'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName',
       'device.mobileDeviceModel', 'device.mobileInputSelector',
       'device.operatingSystem', 'device.operatingSystemVersion',
       'device.screenColors', 'device.screenResolution', 'geoNetwork.city',
       'geoNetwork.cityId', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.networkLocation',
       'geoNetwork.region', 'geoNetwork.subContinent', 'totals.bounces',
       'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'totals.ses

In [11]:
#creating new df with just the column I need and dropping old to save RAM
print(df_small['trafficSource.campaign'].unique())
print(len(df_small[df_small['trafficSource.campaign'] == "(not set)"]) - 100000)

['(not set)' 'Data Share Promo' 'Run of Network Line Item'
 'Page: contains "/google+redesign/drinkware"'
 'Page: contains "/google+redesign/drinkware" Similar Audiences'
 'AW - Accessories' 'Smart Display Campaign'
 '"google + redesign/Accessories March 17" All Users'
 '"google + redesign/Accessories March 17" All Users Similar Audiences'
 '(direct)' 'AW - YouTube Brand' 'AW - Google Brand' 'AW - Bags'
 'AW - Apparel' 'AW - Office' 'AW - Drinkware' 'Technology/Technophiles'
 'AW - Dynamic Search Ads Whole Site' 'AW - Android Brand'
 'AW - Waze Brand']
-5426


From the first 100,000 lines of the CSV, we only have 5426 rows that contain a campaign

**Ideas:**
Do a number of random samples and get the average